# База данных для Universal Dep

### Установка пакетов

In [1]:
!pip install conllu

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Импорты

In [2]:
import mysql.connector
import os
import re
import conllu
from conllu import parse
from conllu import parse_incr

In [3]:
mydb = mysql.connector.connect(
  host="localhost",
  user="tatiana",
  passwd="P@ssw0rdMySQL",
  database="unidep"
)

In [4]:
cur = mydb.cursor(dictionary=True)

In [5]:
mydb.encoding = 'utf-8'

In [6]:
cur.execute("DROP TABLE IF EXISTS sentences")
cur.execute("CREATE TABLE sentences (sent_id VARCHAR(255) PRIMARY KEY, ru_text TEXT, en_text TEXT)")
cur.execute("DROP TABLE IF EXISTS tokens")
cur.execute("CREATE TABLE tokens (token_id MEDIUMINT PRIMARY KEY AUTO_INCREMENT, sent_id VARCHAR(255), num_in_sentence MEDIUMINT, word_form VARCHAR(255), lemma VARCHAR(255), pos VARCHAR(255), head SMALLINT, space_after SMALLINT)")
cur.execute("DROP TABLE IF EXISTS bigrams")
cur.execute("CREATE TABLE bigrams (bigram_id MEDIUMINT PRIMARY KEY AUTO_INCREMENT, sent_id VARCHAR(255), wordpos1 SMALLINT, wordpos2 SMALLINT, lemma1 VARCHAR(255), lemma2 VARCHAR(255), pos1 VARCHAR(255), pos2 VARCHAR(255))")
# add other fields from conllu 

In [7]:
mydb.commit()

In [8]:
cur.execute("SHOW TABLES")

for x in cur:
  print(x)

{'Tables_in_unidep': 'bigrams'}
{'Tables_in_unidep': 'sentences'}
{'Tables_in_unidep': 'tokens'}


### Построение и заполнение базы данных

In [9]:
sentences = []
tokens = []

data_file = open("ru_pud-ud-test.conllu", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    id_line, rus_line, en_line = "", "", ""
    lines = [line for line in tokenlist.serialize().split("\n") if line.startswith("#")]
    for line in lines:
        if "sent_id" in line:
            id_line = line
        elif "english_text" in line:
            en_line = line
        elif "text" in line:
            rus_line = line
    ID = id_line[12:]
    ru_text = rus_line[9:]
    en_text = en_line[17:]
    sentences.append((ID, ru_text, en_text),)
    for token in tokenlist:
        space_after = 1
        if token["misc"] is not None:
            if "SpaceAfter" in token["misc"]:
                space_after = 0
        tokens.append((ID, token["id"], token["form"], token["lemma"].lower(), token["upostag"], token["head"], space_after),)

In [10]:
stmt = "INSERT INTO sentences (sent_id, ru_text, en_text) VALUES (%s, %s, %s)"
cur.executemany(stmt, sentences)

stmt = "INSERT INTO tokens (sent_id, num_in_sentence, word_form, lemma, pos, head, space_after) VALUES (%s, %s, %s, %s, %s, %s, %s)"
cur.executemany(stmt, tokens)

In [11]:
mydb.commit()

In [12]:
bigrams = []

data_file = open("ru_pud-ud-test.conllu", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    lines = [line for line in tokenlist.serialize().split("\n") if line.startswith("#")]
    for line in lines:
        if "sent_id" in line:
            id_line = line
    ID = id_line[12:]
    tokenlist = [token for token in tokenlist if token["upostag"] != "PUNCT"]
    for i in range(1, len(tokenlist)):
        tok1 = tokenlist[i-1]
        tok2 = tokenlist[i]
        bigrams.append((ID, tok1["id"], tok2["id"], tok1["lemma"].lower(), tok2["lemma"].lower(), tok1["upostag"], tok2["upostag"]),)


In [13]:
stmt = "INSERT INTO bigrams (sent_id, wordpos1, wordpos2, lemma1, lemma2, pos1, pos2) VALUES (%s, %s, %s, %s, %s, %s, %s)"
cur.executemany(stmt, bigrams)

In [14]:
mydb.commit()

### Заполнение базы данных

In [15]:
def print_res(res_list, query, method="lemma"):
    if not res_list:
        return ""
    
    out_list = []
    sent = ""
    sent_id = res_list[0]["sent_id"]
    for group in res_list:
        if group["sent_id"] != sent_id:
            out_list.append(sent[:-1])
            sent = ""
            sent_id = group["sent_id"]
        if group[method] == query:
            sent += "<b>"
        sent += group["word_form"]
        if group[method] == query:
            sent += "</b>"
        if group["space_after"]:
            sent += " "
    out_list.append(sent[:-1])
    
    return out_list

In [16]:
lemma = "сша"
stmt = "SELECT t.sent_id, t.word_form, t.lemma, t.space_after FROM (SELECT DISTINCT sent_id FROM tokens WHERE lemma='{}') AS s INNER JOIN tokens AS t ON t.sent_id = s.sent_id".format(lemma)
cur.execute(stmt)
print_res(cur.fetchall(), lemma, method="lemma")

['«Если передача цифровых технологий сегодня в <b>США</b> происходит впервые, то о мирной передаче власти такого не скажешь», – написала Кори Шульман, специальный помощник президента Обамы в своем блоге в понедельник.',
 'Максимально допустимая сумма на одного человека — 5000 долларов <b>США</b>.',
 'Для сравнения, строительство подземной станции метро НоМа (NoMa), которая открылась в 2004 году, обошлось в 103,7 миллиона долларов <b>США</b>.',
 'Г-н Осборн поступил на работу в агентство спикеров <b>США</b> после того, как его уволили в июле.',
 'Инвестиции в эту область в 2015 году выросли на 6% и достигли 221 миллиарда долларов <b>США</b>.',
 'В настоящее время максимальный размер штрафа, который Совет по недвижимости Онтарио (RECO) может наложить на агента, составляет 25000 долларов <b>США</b>.',
 'Руководители также получили так называемую «сдельную заработную плату» за выполнение или превышение ожиданий, разделив между собой сумму в 1,5 миллиона долларов <b>США</b>, или в среднем 1

In [17]:
upos = "PRON"
stmt = "SELECT t.sent_id, t.word_form, t.pos, t.space_after FROM (SELECT DISTINCT sent_id FROM tokens WHERE pos='{}') AS s INNER JOIN tokens AS t ON t.sent_id = s.sent_id".format(upos)
cur.execute(stmt)
print_res(cur.fetchall(), upos, method="pos")

['«Если передача цифровых технологий сегодня в США происходит впервые, то о мирной передаче власти <b>такого</b> не скажешь», – написала Кори Шульман, специальный помощник президента Обамы в своем блоге в понедельник.',
 'Для тех, <b>кто</b> следит за передачей всех материалов, появившихся в социальных сетях о Конгрессе, <b>это</b> будет происходить несколько по-другому.',
 'Но в отступлении от риторики прошлого о сокращении иммиграции кандидат Республиканской партии заявил, что в качестве президента <b>он</b> позволил бы въезд «огромного количества» легальных мигрантов на основе «системы заслуг».',
 '«<b>Мне</b> вовсе не хочется оказывать на <b>вас</b> какое-либо давление, но судьба республики зависит от <b>вас</b>», – сказал <b>он</b> толпе, собравшейся на спортивной площадке в Университете Северной Каролины.',
 '<b>То</b>, что <b>она</b> говорит и что <b>она</b> делает, на самом деле невероятно.',
 'Вместо <b>того</b> чтобы изучать научный метод абстрактно, студенты постигают его су

In [18]:
lemma = "сша"
stmt = "SELECT sentences.ru_text, sentences.en_text FROM sentences, (SELECT DISTINCT sent_id FROM tokens WHERE lemma='{}') AS q WHERE sentences.sent_id = q.sent_id".format(lemma)
cur.execute(stmt)
cur.fetchall()

[{'ru_text': '«Если передача цифровых технологий сегодня в США происходит впервые, то о мирной передаче власти такого не скажешь», – написала Кори Шульман, специальный помощник президента Обамы в своем блоге в понедельник.',
  'en_text': '“While much of the digital transition is unprecedented in the United States, the peaceful transition of power is not,” Obama special assistant Kori Schulman wrote in a blog post Monday.'},
 {'ru_text': 'Максимально допустимая сумма на одного человека — 5000 долларов США.',
  'en_text': '$5,000 per person, the maximum allowed.'},
 {'ru_text': 'Для сравнения, строительство подземной станции метро НоМа (NoMa), которая открылась в 2004 году, обошлось в 103,7 миллиона долларов США.',
  'en_text': 'By comparison, it cost $103.7 million to build the NoMa infill Metro station, which opened in 2004.'},
 {'ru_text': 'Г-н Осборн поступил на работу в агентство спикеров США после того, как его уволили в июле.',
  'en_text': 'Mr Osborne signed up with a US speakers

In [19]:
upos = "PRON"
stmt = "SELECT sentences.ru_text, sentences.en_text FROM sentences, (SELECT DISTINCT sent_id FROM tokens WHERE pos='{}') AS q WHERE sentences.sent_id = q.sent_id".format(upos)
cur.execute(stmt)
cur.fetchall()

[{'ru_text': '«Если передача цифровых технологий сегодня в США происходит впервые, то о мирной передаче власти такого не скажешь», – написала Кори Шульман, специальный помощник президента Обамы в своем блоге в понедельник.',
  'en_text': '“While much of the digital transition is unprecedented in the United States, the peaceful transition of power is not,” Obama special assistant Kori Schulman wrote in a blog post Monday.'},
 {'ru_text': 'Для тех, кто следит за передачей всех материалов, появившихся в социальных сетях о Конгрессе, это будет происходить несколько по-другому.',
  'en_text': 'For those who follow social media transitions on Capitol Hill, this will be a little different.'},
 {'ru_text': 'Но в отступлении от риторики прошлого о сокращении иммиграции кандидат Республиканской партии заявил, что в качестве президента он позволил бы въезд «огромного количества» легальных мигрантов на основе «системы заслуг».',
  'en_text': 'But in a break from his past rhetoric about curtailing 

In [20]:
from collections import defaultdict

In [21]:
def print_pair(res_list, query1, query2, method="lemma"):
    res = defaultdict(list)
    
    if not res_list:
        return ""
    
    out_list = []
    
    for i in res_list:
        res[i['sent_id']].append(i)
    
    for sent_id in res:
        res_list = res[sent_id]
        sentpieces = [[tok["word_form"] + " " * tok["space_after"], 0] for tok in res_list]
        for i, group_pair in enumerate(zip(res_list, res_list[1:])):
            if group_pair[0][method] == query1 and group_pair[1][method] == query2:
                sentpieces[i][1] = 1
                sentpieces[i+1][1] = 1
        sent = ""
        for tok in sentpieces:
            if tok[1]:
                sent += "<b>"
            sent += tok[0]
            if tok[1]:
                sent += "</b>"
        out_list.append(sent[:-1])
    
    return out_list

In [22]:
lemma1 = "в"
lemma2 = "сша"
stmt = "SELECT t.sent_id, t.word_form, t.lemma, t.space_after FROM (SELECT DISTINCT sent_id FROM bigrams WHERE lemma1='{}' AND lemma2='{}') AS s INNER JOIN tokens AS t ON t.sent_id = s.sent_id".format(lemma1, lemma2)
cur.execute(stmt)
print_pair(cur.fetchall(), lemma1, lemma2, method="lemma")

['«Если передача цифровых технологий сегодня <b>в </b><b>США </b>происходит впервые, то о мирной передаче власти такого не скажешь», – написала Кори Шульман, специальный помощник президента Обамы в своем блоге в понедельник.',
 '<b>В </b><b>США </b>фьючерсные сделки с акциями поднялись более чем на 1%, так же как и на европейских рынках.',
 'Альбом «Really Really Love You» был выпущен в августе и вошел в топ-50, сингл из этого альбома «Shaky Ground» появился в сентябре, но Гейер уже была <b>в </b><b>США</b>.']

In [23]:
pos1 = "NOUN"
pos2 = "NOUN"
stmt = "SELECT t.sent_id, t.word_form, t.pos, t.space_after FROM (SELECT DISTINCT sent_id FROM bigrams WHERE pos1='{}' AND pos2='{}') AS s INNER JOIN tokens AS t ON t.sent_id = s.sent_id".format(pos1, pos2)
cur.execute(stmt)
print_pair(cur.fetchall(), pos1, pos2, method="pos")

['«Если передача цифровых технологий сегодня в США происходит впервые, то о мирной <b>передаче </b><b>власти </b>такого не скажешь», – написала Кори Шульман, специальный <b>помощник </b><b>президента </b>Обамы в своем блоге в понедельник.',
 'Но в отступлении от <b>риторики </b><b>прошлого </b>о <b>сокращении </b><b>иммиграции </b><b>кандидат </b>Республиканской партии заявил, что в <b>качестве </b><b>президента </b>он позволил бы въезд «огромного количества» легальных мигрантов на основе «<b>системы </b><b>заслуг</b>».',
 '«Мне вовсе не хочется оказывать на вас какое-либо давление, но <b>судьба </b><b>республики </b>зависит от вас», – сказал он толпе, собравшейся на спортивной площадке в Университете Северной Каролины.',
 'В <b>начале </b><b>октября </b>переходная команда использовала то же место для встреч с технологическими лоббистами, с <b>приглашением </b><b>представителей </b>из Убер, Американской <b>Ассоциации </b><b>Кинокомпаний</b>, <b>Ассоциации </b><b>производителей </b>быто

In [24]:
lemma1 = "в"
lemma2 = "сша"
stmt = "SELECT sentences.ru_text, sentences.en_text FROM sentences, (SELECT DISTINCT sent_id FROM bigrams WHERE lemma1='{}' AND lemma2='{}') AS q WHERE sentences.sent_id = q.sent_id".format(lemma1, lemma2)
cur.execute(stmt)
cur.fetchall()

[{'ru_text': '«Если передача цифровых технологий сегодня в США происходит впервые, то о мирной передаче власти такого не скажешь», – написала Кори Шульман, специальный помощник президента Обамы в своем блоге в понедельник.',
  'en_text': '“While much of the digital transition is unprecedented in the United States, the peaceful transition of power is not,” Obama special assistant Kori Schulman wrote in a blog post Monday.'},
 {'ru_text': 'В США фьючерсные сделки с акциями поднялись более чем на 1%, так же как и на европейских рынках.',
  'en_text': 'U.S. stock futures are surging by more than 1%, alongside European markets.'},
 {'ru_text': 'Альбом «Really Really Love You» был выпущен в августе и вошел в топ-50, сингл из этого альбома «Shaky Ground» появился в сентябре, но Гейер уже была в США.',
  'en_text': 'Really Really Love You was released in August and reach the top 50; "Shaky Ground", the related single, appeared in September but Geyer was already in the US.'}]

In [25]:
pos1 = "NOUN"
pos2 = "NOUN"
stmt = "SELECT sentences.ru_text, sentences.en_text FROM sentences, (SELECT DISTINCT sent_id FROM bigrams WHERE pos1='{}' AND pos2='{}') AS q WHERE sentences.sent_id = q.sent_id".format(pos1, pos2)
cur.execute(stmt)
cur.fetchall()

[{'ru_text': '«Если передача цифровых технологий сегодня в США происходит впервые, то о мирной передаче власти такого не скажешь», – написала Кори Шульман, специальный помощник президента Обамы в своем блоге в понедельник.',
  'en_text': '“While much of the digital transition is unprecedented in the United States, the peaceful transition of power is not,” Obama special assistant Kori Schulman wrote in a blog post Monday.'},
 {'ru_text': 'Но в отступлении от риторики прошлого о сокращении иммиграции кандидат Республиканской партии заявил, что в качестве президента он позволил бы въезд «огромного количества» легальных мигрантов на основе «системы заслуг».',
  'en_text': 'But in a break from his past rhetoric about curtailing immigration, the GOP nominee proclaimed that as president he would allow “tremendous numbers” of legal immigrants based on a “merit system.”'},
 {'ru_text': '«Мне вовсе не хочется оказывать на вас какое-либо давление, но судьба республики зависит от вас», – сказал он

### Flask-приложение

In [26]:
import os
from flask import Flask, Markup
from flask import url_for, render_template, request, redirect

In [27]:
style_eng = """table {
  border-collapse: collapse;
  width: 100%;
}

table * {
  text-align: center;
}

td, th {
  border: 1px solid #dddddd;
  padding: 8px;
}

tr:nth-child(4n) {
  background-color: #dddddd;
}

tr:nth-child(4n+1) {
  background-color: #dddddd;
}"""

In [28]:
style_noeng = """table {
  border-collapse: collapse;
  width: 100%;
}

table * {
  text-align: center;
}

td, th {
  border: 1px solid #dddddd;
  padding: 8px;
}

tr:nth-child(odd) {
  background-color: #dddddd;
}"""

In [29]:
app = Flask(__name__)

In [30]:
@app.route('/search')
def printsearch(query, results, eng=False):
    results = [Markup(res) for res in results]
    if eng:
        style = style_eng
    else:
        style = style_noeng
    return render_template('index.html', searched=True, style=style, query=query, results=results)

In [31]:
@app.route('/search')
def printbigram(query1, query2, results, eng=False):
    results = [Markup(res) for res in results]
    if eng:
        style = style_eng
    else:
        style = style_noeng
    return render_template('bigrams.html', searched=True, style=style, query1=query1, query2=query2, results=results)

In [32]:
def add_eng_lemma(results, lemma):
    trim_results = [re.sub(r"</?b>", "", r) for r in results]
    stmt = "SELECT sentences.ru_text, sentences.en_text FROM sentences, (SELECT DISTINCT sent_id FROM tokens WHERE lemma='{}') AS q WHERE sentences.sent_id = q.sent_id".format(lemma)
    cur.execute(stmt)
    res = cur.fetchall()
    en_texts = [r["en_text"] for r in res]
    twi_res = []
    for result in zip(results, en_texts):
        twi_res.append(result[0])
        twi_res.append(result[1])
    return twi_res

In [33]:
def add_eng_upos(results, upos):
    trim_results = [re.sub(r"</?b>", "", r) for r in results]
    stmt = "SELECT sentences.ru_text, sentences.en_text FROM sentences, (SELECT DISTINCT sent_id FROM tokens WHERE pos='{}') AS q WHERE sentences.sent_id = q.sent_id".format(upos)
    cur.execute(stmt)
    res = cur.fetchall()
    en_texts = [r["en_text"] for r in res]
    twi_res = []
    for result in zip(results, en_texts):
        twi_res.append(result[0])
        twi_res.append(result[1])
    return twi_res

In [34]:
def add_eng_lemma_bigrams(results, lemma1, lemma2):
    trim_results = [re.sub(r"</?b>", "", r) for r in results]
    stmt = "SELECT sentences.ru_text, sentences.en_text FROM sentences, (SELECT DISTINCT sent_id FROM bigrams WHERE lemma1='{}' AND lemma2='{}') AS q WHERE sentences.sent_id = q.sent_id".format(lemma1, lemma2)
    cur.execute(stmt)
    res = cur.fetchall()
    en_texts = [r["en_text"] for r in res]
    twi_res = []
    for result in zip(results, en_texts):
        twi_res.append(result[0])
        twi_res.append(result[1])
    return twi_res

In [35]:
def add_eng_pos_bigrams(results, pos1, pos2):
    trim_results = [re.sub(r"</?b>", "", r) for r in results]
    stmt = "SELECT sentences.ru_text, sentences.en_text FROM sentences, (SELECT DISTINCT sent_id FROM bigrams WHERE pos1='{}' AND pos2='{}') AS q WHERE sentences.sent_id = q.sent_id".format(pos1, pos2)
    cur.execute(stmt)
    res = cur.fetchall()
    en_texts = [r["en_text"] for r in res]
    twi_res = []
    for result in zip(results, en_texts):
        twi_res.append(result[0])
        twi_res.append(result[1])
    return twi_res

In [36]:
@app.route('/')
def index():
    eng = False
    if request.args:
        query = request.args["query"]
        if request.args["method"] == "lemma":
            lemma = query.lower()
            stmt = "SELECT t.sent_id, t.word_form, t.lemma, t.space_after FROM (SELECT DISTINCT sent_id FROM tokens WHERE lemma='{}') AS s INNER JOIN tokens AS t ON t.sent_id = s.sent_id".format(lemma)
            cur.execute(stmt)
            res = cur.fetchall()
            results = print_res(res, lemma, method="lemma")
            if "eng" in request.args:
                eng = True
                results = add_eng_lemma(results, lemma)
        else:
            upos = query
            stmt = "SELECT t.sent_id, t.word_form, t.pos, t.space_after FROM (SELECT DISTINCT sent_id FROM tokens WHERE pos='{}') AS s INNER JOIN tokens AS t ON t.sent_id = s.sent_id".format(upos)
            cur.execute(stmt)
            res = cur.fetchall()
            results = print_res(res, upos, method="pos")
            if "eng" in request.args:
                eng = True
                results = add_eng_upos(results, upos)
        return printsearch(query, results, eng)
    return render_template('index.html', searched=False)

In [37]:
@app.route('/bigrams')
def bigrams():
    eng = False
    if request.args:
        query1 = request.args["query1"]
        query2 = request.args["query2"]
        if request.args["method"] == "lemma":
            lemma1 = query1.lower()
            lemma2 = query2.lower()
            stmt = "SELECT t.sent_id, t.word_form, t.lemma, t.space_after FROM (SELECT DISTINCT sent_id FROM bigrams WHERE lemma1='{}' AND lemma2='{}') AS s INNER JOIN tokens AS t ON t.sent_id = s.sent_id".format(lemma1, lemma2)
            cur.execute(stmt)
            res = cur.fetchall()
            results = print_pair(res, lemma1, lemma2, method="lemma")
            if "eng" in request.args:
                eng = True
                results = add_eng_lemma_bigrams(results, lemma1, lemma2)
        else:
            pos1 = query1
            pos2 = query2
            stmt = "SELECT t.sent_id, t.word_form, t.pos, t.space_after FROM (SELECT DISTINCT sent_id FROM bigrams WHERE pos1='{}' AND pos2='{}') AS s INNER JOIN tokens AS t ON t.sent_id = s.sent_id".format(pos1, pos2)
            cur.execute(stmt)
            res = cur.fetchall()
            results = print_pair(res, pos1, pos2, method="pos")
            if "eng" in request.args:
                eng = True
                results = add_eng_pos_bigrams(results, pos1, pos2)
        return printbigram(query1, query2, results, eng)
    return render_template('bigrams.html', searched=False)

In [38]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jan/2020 20:18:26] "GET /bigrams?query1=в&query2=США&method=lemma&eng=on HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2020 20:18:30] "GET /bigrams?query1=в&query2=США&method=lemma HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2020 20:18:45] "GET /bigrams?query1=в&query2=США&method=lemma&eng=on HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2020 20:18:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2020 20:18:54] "GET /?query=США&method=lemma&eng=on HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2020 20:19:01] "GET /?query=PRON&method=upos&eng=on HTTP/1.1" 200 -
